In [31]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('dataset.csv')

# Preprocess the data
df['TotalTime'] = df['TotalTime'].str.extract(r'(\d+)')
df['AvgRating'] = df['AvgRating'].fillna(0)  # Fill missing ratings with 0


# Convert NumSteps to numeric
df['NumSteps'] = pd.to_numeric(df['NumSteps'], errors='coerce')

# Fill missing values with interpolation
df = df.interpolate()

# Preview the preprocessed data
print(df.head())

      ID                                   Title  \
0  10000                            Tomato Paste   
1  10001                                Aloo Dum   
2  10002                         Caulinana salad   
3  10003  Spicy Corn Salad With Avocado Dressing   
4  10004                             Ginger Beer   

                                      IngredientsRaw TotalTime  NumSteps  \
0      48 large tomatoes#item, 2 teaspoons salt#item         3       8.0   
1  1⁄2 kg potato (small and round)#item, oil (for...         4      18.0   
2  1 small cauliflower#item, 2 bananas#item, 2 ta...        15      11.0   
3  4 cups whole kernel corn (I use frozen)#item, ...        12      10.0   
4  TO MAKE THE GINGER BEER#item, 1 1⁄2 teaspoons ...       312      18.0   

   AvgRating  NumReviews                          RecipeURL  
0       3.00    2.000000  https://www.food.com/recipe/10000  
1       0.00    2.333333  https://www.food.com/recipe/10001  
2       0.00    2.666667  https://www.food.c

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Read the user-item matrix dataset from a CSV file
user_item_matrix = pd.read_csv("user-item-metrix.csv")

# Read the recipe dataset from a CSV file
recipe_data = pd.read_csv("dataset.csv")

# Assuming you have a user's preferred ingredients
user_ingredients = ["Tomato", "Salt", "Garlic"]

# Fill missing values in the user-item matrix with 0
user_item_matrix.fillna(0, inplace=True)

# Calculate user similarity matrix using cosine similarity
user_similarity = linear_kernel(user_item_matrix, user_item_matrix)

# Get the most similar users for the target user
target_user_id = 11  # Replace with the target user ID
target_user_index = user_item_matrix.index.get_loc(target_user_id)
similar_users_indices = user_similarity[target_user_index].argsort()[::-1]

# Preprocess recipe ingredients: convert to lowercase and remove unwanted characters
recipe_data["IngredientsProcessed"] = recipe_data["IngredientsRaw"].fillna("").astype(str).str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))

# Apply TF-IDF vectorization to convert text into numerical features
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(recipe_data["IngredientsProcessed"])

# Preprocess user ingredients: convert to lowercase and remove unwanted characters
user_ingredients_processed = [re.sub(r'[^a-z\s]', '', ingredient.lower()) for ingredient in user_ingredients]

# Calculate the cosine similarity between the user's ingredient vector and all recipes
user_vector = vectorizer.transform([" ".join(user_ingredients_processed)])
cosine_similarities = linear_kernel(user_vector, tfidf_matrix).flatten()

print(cosine_similarities)

# Combine the user-based and content-based similarity scores
combined_scores = user_similarity[target_user_index] * cosine_similarities

# Get indices of recipes sorted by combined similarity
similar_recipe_indices = combined_scores.argsort()[::-1]

# Recommend recipes based on combined similarity
num_recommendations = 5  # Specify the number of recommendations needed
recommendations = recipe_data.iloc[similar_recipe_indices][:num_recommendations]

# Display the recommended recipes
for _, recipe in recommendations.iterrows():
    print("Recipe Name:", recipe["Title"])
    print("Ingredients Needed:", recipe["IngredientsRaw"])
    print("Total Time to Prepare:", recipe["TotalTime"])
    print("Average Rating:", recipe["AvgRating"])
    print()

[0.         0.         0.         ... 0.         0.         0.06072066]
Recipe Name: Cheesy Red-White Sauce
Ingredients Needed: 2 tablespoons butter#item, 2 tablespoons flour#item, 1 cup cream#item, 1 cup tomato juice#item, salt & pepper#item, 1 1⁄2 cups grated cheese#item
Total Time to Prepare: 20mins
Average Rating: nan

Recipe Name: Green Beans in Tomato Salsa
Ingredients Needed: 1 tablespoon olive oil#item, 1 clove garlic, crushed#item, 1 kg fresh green beans#item, 1 can diced peeled tomatoes#item, 1⁄2 onion, chopped#item, 2 tablespoons tomato paste#item, salt and pepper#item
Total Time to Prepare: 30mins
Average Rating: 5.0

Recipe Name: Spaghetti Sauce
Ingredients Needed: 2 tablespoons olive oil#item, 3⁄4 cup chopped onion#item, 3 garlic cloves, crushed#item, 2 (6 ounce) cans tomato paste#item, 2 1⁄2 cups water#item, 2 (8 ounce) cans tomato sauce#item, 1 teaspoon oregano#item, 1 teaspoon basil#item, 1⁄2 teaspoon sugar#item, salt & pepper#item
Total Time to Prepare: 1hr 15mins
Ave

In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Read the user-item matrix dataset from a CSV file
user_item_matrix = pd.read_csv("user-item-metrix.csv")

# Read the recipe dataset from a CSV file
recipe_data = pd.read_csv("dataset.csv")

# Assuming you have a user's preferred ingredients
user_ingredients = ["Tomato", "Salt", "Garlic"]

# Fill missing values in the user-item matrix with 0
user_item_matrix.fillna(0, inplace=True)

# Calculate user similarity matrix using cosine similarity
user_similarity = linear_kernel(user_item_matrix, user_item_matrix)

# Get the most similar users for the target user
target_user_id = 1  # Replace with the target user ID
target_user_index = user_item_matrix.index.get_loc(target_user_id)

# Preprocess recipe ingredients: convert to lowercase and remove unwanted characters
recipe_data["IngredientsProcessed"] = recipe_data["IngredientsRaw"].fillna("").astype(str).str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))

# Apply TF-IDF vectorization to convert text into numerical features
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(recipe_data["IngredientsProcessed"])

# Preprocess user ingredients: convert to lowercase and remove unwanted characters
user_ingredients_processed = [re.sub(r'[^a-z\s]', '', ingredient.lower()) for ingredient in user_ingredients]

# Calculate the cosine similarity between the user's ingredient vector and all recipes
user_vector = vectorizer.transform([" ".join(user_ingredients_processed)])
cosine_similarities = linear_kernel(user_vector, tfidf_matrix).flatten()

print(cosine_similarities)

target_user_similarity = user_similarity[target_user_index]
# Combine the user-based and content-based similarity scores
combined_scores = target_user_similarity * cosine_similarities

# Get indices of recipes sorted by combined similarity
similar_recipe_indices = combined_scores.argsort()[::-1]

# Recommend recipes based on combined similarity
num_recommendations = 5  # Specify the number of recommendations needed
recommendations = recipe_data.iloc[similar_recipe_indices][:num_recommendations]

# Display the recommended recipes
for _, recipe in recommendations.iterrows():
    print("Recipe Name:", recipe["Title"])
    print("Ingredients Needed:", recipe["IngredientsRaw"])
    print("Total Time to Prepare:", recipe["TotalTime"])
    print("Average Rating:", recipe["AvgRating"])
    print()


[0.         0.         0.         ... 0.         0.         0.06072066]
Recipe Name: Cheesy Red-White Sauce
Ingredients Needed: 2 tablespoons butter#item, 2 tablespoons flour#item, 1 cup cream#item, 1 cup tomato juice#item, salt & pepper#item, 1 1⁄2 cups grated cheese#item
Total Time to Prepare: 20mins
Average Rating: nan

Recipe Name: Green Beans in Tomato Salsa
Ingredients Needed: 1 tablespoon olive oil#item, 1 clove garlic, crushed#item, 1 kg fresh green beans#item, 1 can diced peeled tomatoes#item, 1⁄2 onion, chopped#item, 2 tablespoons tomato paste#item, salt and pepper#item
Total Time to Prepare: 30mins
Average Rating: 5.0

Recipe Name: Spaghetti Sauce
Ingredients Needed: 2 tablespoons olive oil#item, 3⁄4 cup chopped onion#item, 3 garlic cloves, crushed#item, 2 (6 ounce) cans tomato paste#item, 2 1⁄2 cups water#item, 2 (8 ounce) cans tomato sauce#item, 1 teaspoon oregano#item, 1 teaspoon basil#item, 1⁄2 teaspoon sugar#item, salt & pepper#item
Total Time to Prepare: 1hr 15mins
Ave